## Import library

In [12]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Import datasets

In [13]:
df1 = df = pd.read_excel('../datasets/Online Retail.xlsx')

## Menampilkan data urutan teratas

In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Cleansing Data

In [15]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [16]:
# Menghapus spasi pada kolom Description dan menghapus baris yang memiliki nilai kosong/tidak valid pada kolom InvoiceNo
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

In [17]:
# Menghapus transaksi yang dibeli secara tidak langsung (C)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [18]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Mengatur transaksi berdasarkan negara dan invoice

In [19]:

basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [20]:
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Mengubah data kuantitas menjadi 1 / 0

In [21]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

## Mencari Frequent Itemsets

In [32]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

c:\Users\Budi\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [33]:
frequent_itemsets

,support,itemsets
0,0.030612,(10 COLOUR SPACEBOY PEN)
1,0.015306,(12 COLOURED PARTY BALLOONS)
2,0.015306,(12 PENCIL SMALL TUBE WOODLAND)
3,0.017857,(12 PENCILS SMALL TUBE RED RETROSPOT)
4,0.017857,(12 PENCILS SMALL TUBE SKULL)
...,...,...
39681,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND, LUNCH BOX..."
39682,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND, LUNCH BOX..."
39683,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND, LUNCH BOX..."
39684,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND, LUNCH BOX..."


## Mencari Association Rule

In [34]:

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [35]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CARD DOLLY GIRL),(10 COLOUR SPACEBOY PEN),0.030612,0.030612,0.010204,0.333333,10.888889,0.009267,1.454082
1,(10 COLOUR SPACEBOY PEN),(CARD DOLLY GIRL),0.030612,0.030612,0.010204,0.333333,10.888889,0.009267,1.454082
2,(CHARLOTTE BAG SUKI DESIGN),(10 COLOUR SPACEBOY PEN),0.043367,0.030612,0.010204,0.235294,7.686275,0.008877,1.267661
3,(10 COLOUR SPACEBOY PEN),(CHARLOTTE BAG SUKI DESIGN),0.030612,0.043367,0.010204,0.333333,7.686275,0.008877,1.434949
4,(ICE CREAM SUNDAE LIP GLOSS),(10 COLOUR SPACEBOY PEN),0.045918,0.030612,0.010204,0.222222,7.259259,0.008798,1.246356
